## Carga librerias necesarias


In [ ]:
import pandas as pd 
import numpy as np
import os
from pandas import concat

## Carga del los dataset correspondientes al año 2022
Se cargan los csv de cada mes en una lista

In [ ]:
file_names = os.listdir('../data/aire/')
file_names

## Crear único dataset con los los datos del año 2022
Recorrer la lista files_names para crear un único dataset


In [ ]:
df_aire = pd.DataFrame()
for file_name in file_names:
    df_current = pd.read_csv('../data/aire/' + file_name, delimiter=';')
    print(file_name, df_current.shape)
    df_aire = pd.concat([df_aire, df_current], ignore_index=True)
    

## Filtrar los datos con el componente NO2

In [2]:
df_aire = df_aire[(df_aire['MAGNITUD'] == 9)]
df_aire.head()

NameError: name 'df_aire' is not defined

## Crear una nueva columna FECHA 

In [ ]:
df_aire['FECHA'] = pd.to_datetime(df_aire[['ANO','MES','DIA']].rename(columns = {'ANO': 'YEAR', 'MES': 'MONTH', 'DIA': 'DAY'}))

## Extraer las datos de las medidas tomadas

In [ ]:
data_pm2_5_columns = [ 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24'] # feature
veri_columns = [ 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24']

hora_column = np.arange(1,25) # feature label

## Crear un dataset auxiliar con los datos extraidos  y las nuevas columnas creadas

In [ ]:


data_pm2_5 = pd.DataFrame(columns=['hora', 'dato', 'verificado'])
aux = pd.DataFrame(columns=['ano', 'mes', 'dia', 'hora', 'dato', 'verificado', 'estacion']) # aux de próxima fila a ser añadida
for  (h,d,v) in zip(hora_column, data_pm2_5_columns, veri_columns): 
     aux['dato'] = df_aire [ data_pm2_5_columns[h-1] ] 
     aux['verificado'] = df_aire [ veri_columns[h-1] ] 
     aux['hora'] = h-1
     aux[['ano', 'mes', 'dia']] = df_aire[['ANO','MES','DIA']]
     aux['estacion'] = df_aire ['ESTACION']
     data_pm2_5 = pd.concat([data_pm2_5, aux], ignore_index=True)
data_pm2_5 = data_pm2_5.astype({'hora': 'int32'}, copy = False)

data_pm2_5.head()



## Añadir la columna FECHA en formato datetime al nuevo dataset 

In [ ]:
data_pm2_5['FECHA'] = pd.to_datetime(data_pm2_5[['ano','mes','dia','hora']].rename(columns = {'ano': 'year', 'mes': 'month','dia': 'day', 'hora': 'hour'}))
data_pm2_5.head()

## Multi-index data frame con estacion y FECHA :O

In [ ]:
data_pm2_5.set_index(['estacion', 'FECHA'], inplace=True)
data_pm2_5.head()

## Ordenar por estación y FECHA 

In [ ]:
data_pm2_5 = data_pm2_5.sort_values(['estacion', 'FECHA'])
data_pm2_5.head()



## Rellenar con NaN las filas perdidas 
 Para cada estación, rellenamos con NaN si hay alguna fila perdida y guardamos el resultado en un nuevo dataframe que ya debería tener todas las filas


In [ ]:
estaciones = data_pm2_5.index.get_level_values('estacion').unique()
datetime_index = pd.date_range(start='1/1/2022', end='12/31/2022 23:00:00', freq='H', name='FECHA')
data_pm2_5_nuevo = pd.DataFrame()
for estacion in estaciones:
    #data_pm2_5_estacion = data_pm2_5[data_pm2_5['estacion'] == estacion]
    data_pm2_5_estacion = data_pm2_5.loc[estacion]
    print(estacion, len(data_pm2_5_estacion))
    data_pm2_5_estacion = data_pm2_5_estacion.reindex(datetime_index)
    data_pm2_5_estacion.reset_index(inplace=True)
    print(estacion, len(data_pm2_5_estacion))
    data_pm2_5_estacion['estacion'] = estacion
    data_pm2_5_nuevo = pd.concat([data_pm2_5_nuevo, data_pm2_5_estacion],ignore_index=True)
    
print(len(data_pm2_5_nuevo))

In [ ]:
# Multi-index data_pm2_5 frame con estacion y FECHA :O
data_pm2_5_nuevo.set_index(['estacion', 'FECHA'], inplace=True)

##  Actualizar  data con el nuevo df:

In [ ]:
data_pm2_5 = data_pm2_5_nuevo
data_pm2_5.info()
data_pm2_5

In [ ]:
# Mostramos los datos que se habían perdidos
missing_data_pm2_5 = data_pm2_5[data_pm2_5['dato'].isna()]
print(f'Cantidad de datos no existentes (perdidos): {len(missing_data_pm2_5)}')
missing_data_pm2_5

In [ ]:
# Rellenamos las columnas conciernientes a la FECHA que faltan en esos datos perdidos excepto la columna 'dato' que ya trataremos más adelante
dt = missing_data_pm2_5.index.get_level_values('FECHA')
data_pm2_5.loc[data_pm2_5['dato'].isna(),'ano'] = dt.year
data_pm2_5.loc[data_pm2_5['dato'].isna(),'mes'] = dt.month
data_pm2_5.loc[data_pm2_5['dato'].isna(),'dia'] = dt.day
data_pm2_5.loc[data_pm2_5['dato'].isna(),'hora'] = dt.hour
data_pm2_5 = data_pm2_5.astype({'hora': 'int32'}, copy = False)

#data_pm2_5.loc[data_pm2_5['dato'].isna(),['ano', 'mes', 'dia', 'hora']] = [dt.year, dt.month, dt.day, dt.hour]
data_pm2_5[data_pm2_5['dato'].isna()]

In [ ]:


# 6. Por último, marcamos los datos no verificados como NaN
no_verificados = data_pm2_5.loc[data_pm2_5['verificado'] != 'V']
print(f'Cantidad de datos existentes pero no válidos:{len(no_verificados)}')
data_pm2_5.loc[data_pm2_5['verificado'] != 'V', ['dato']] = np.nan
data_pm2_5.loc[data_pm2_5['verificado'] != 'V']



In [ ]:
# borramos columna verificado
del data_pm2_5['verificado']
data_pm2_5.info()

In [ ]:


# Primero de todo, deshacemos el multíndice
data_pm2_5.reset_index(inplace=True) # Salvar los datos sin indices de estacion y FECHA



In [ ]:


# 7. Añadimos columna para el día de la semana
#dates = data_pm2_5.index.get_level_values('FECHA').to_pydatetime() # con multiindex
dates = data_pm2_5['FECHA'].dt.to_pydatetime() # con column y sin multiindex
days_of_week = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
data_pm2_5['dia_semana'] = np.array([days_of_week[d.weekday()] for d in dates])



In [ ]:
data_pm2_5 = data_pm2_5.rename(columns={'dato': 'PM2.5'}, copy = False)

In [ ]:


data_pm2_5.info()
data_pm2_5.head()



In [ ]:
data_pm2_5.tail()

In [ ]:
# Serie con solamente los datos de contaminación
datos = data_pm2_5['PM2.5']
datos

In [ ]:
datos.to_csv('data_pm2_5_hist.csv') # Salvado serie solo datos históricos
data_pm2_5.to_csv('data_pm2_5_FECHA.csv') # Salvado con los datos de las FECHAs